## Set the path of where the csv files are located, as well as initial import statements

In [1]:
csv_data_path = "/home/quinton/Documents/Projects/COMET_Prestudy/scripts/collection/formal_scripts/output_data_ocellID_01_2017.csv" # **PLACE CSV FILEPATH HERE**
sep = "\t" # csv delimiter used
small_model = True # use smaller ML model for better performance, less accuracy

In [2]:
from pathlib import Path 
import pandas as pd
import numpy as np

# Gensim imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# visualization
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/quinton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Functions to handle data loading and data writing

In [3]:
def load_data(filename) -> pd.DataFrame:
    filename = Path(filename)
    if not filename.is_file():
        raise FileNotFoundError

    with open(filename.resolve(), 'r', encoding='utf-8') as file:
        data = pd.read_csv(file, sep=sep)
    return data

def load_data_by_year(directory: Path) -> pd.DataFrame:
    for data_file in Path.glob(Path.joinpath(directory, '*.csv')):
            yield load_data(data_file.resolve())

def write_data(filename: Path, data: pd.DataFrame):
    with open(filename.as_uri(), 'w', encoding='utf-8') as f:
        data.to_csv(f, sep=sep)


In [4]:
# obtain the stop words
stopwords = stopwords.words("english")
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [5]:
data = load_data(csv_data_path)['content']
data.head()

0    Welcome to our very new community forum.  | “W...
1    Option1:   Sign up for an account at https://O...
2    Hi, I checked around, and I can’t seem to find...
3    Hi, First of all, thank you for providing such...
4    I came across the Mozilla Location Service her...
Name: content, dtype: object

# Content Preprocessing

In [6]:
def lemmatize(posts: pd.DataFrame, allowed_post_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm" if small_model else "en_core_web_trf", disable=["parser", 'ner'])
    texts_out = []

    for post in posts:
        doc = nlp(post)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_post_tags:
                new_text.append(token.lemma_)
        final = ' '.join(new_text)
        texts_out.append(final)
    return texts_out

lemmatized_data = lemmatize(data.astype(str))
lemmatized_data[0][:10]

'welcome ve'

In [7]:
def tokenize_lemma(posts: list):
    final = []
    for post in posts:
        new = gensim.utils.simple_preprocess(post, deacc=True)
        final.append(new)
    return final

tokenized_data = tokenize_lemma(lemmatized_data)
tokenized_data[0][:10]

['welcome',
 'very',
 'new',
 'community',
 'forum',
 'good',
 'way',
 'begin',
 'say',
 'introduce']

# Generate id2word dictionary

In [8]:
id2word = corpora.Dictionary(tokenized_data)

corpus = []

for text in tokenized_data:
    new = id2word.doc2bow(text)
    corpus.append(new)

corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 2),
 (9, 1)]

# LDA Visualization

In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=30,
random_state=100, update_every=1, chunksize=100, passes=10, alpha="auto")

In [10]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/quinton/Documents/Projects/COMET_Prestudy/scripts/analysis/.venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarn

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.048001  0.334543       1        1  11.568943
5      0.344514 -0.137942       2        1  10.720208
13    -0.350269  0.183634       3        1   7.721593
28    -0.142528  0.304752       4        1   6.930021
18     0.127617 -0.332763       5        1   5.624039
4      0.260735  0.157396       6        1   5.602596
14     0.129666  0.238130       7        1   5.583468
11    -0.316014 -0.192537       8        1   5.468229
29    -0.250236  0.068511       9        1   3.981192
8      0.201024  0.219072      10        1   3.873947
12     0.275368  0.044649      11        1   3.502666
26    -0.046338  0.274392      12        1   3.372892
22    -0.283865 -0.072509      13        1   3.322507
9     -0.177992  0.173731      14        1   3.211560
21    -0.108338 -0.267268      15        1   2.742958
25    -0.141588  0.072412      16        1   2.051466
19    -0.013061  0.134073      17        1   2.007464
17    -0.140390 -0.179011      18        1   1.913717
0      0.198004 -0.188512      19        1   1.910892
27     0.224351 -0.042737      20        1   1.857388
10    -0.139479 -0.045128      21        1   1.594167
15     0.053440 -0.209668      22        1   1.241452
1      0.102165  0.020178      23        1   1.228940
6     -0.012762 -0.168871      24        1   0.806502
20    -0.026724 -0.064348      25        1   0.612773
16     0.012543 -0.008218      26        1   0.478887
23     0.070236 -0.085083      27        1   0.457054
7      0.045215 -0.112644      28        1   0.454069
3      0.029358 -0.060768      29        1   0.102275
24     0.027345 -0.057462      30        1   0.056132, topic_info=         Term        Freq       Total Category  logprob  loglift
178     tower   89.000000   89.000000  Default  30.0000  30.0000
76       cell  114.000000  114.000000  Default  29.0000  29.0000
93   location   62.000000   62.000000  Default  28.0000  28.0000
158     phone   43.000000   43.000000  Default  27.0000  27.0000
142      help   41.000000   41.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
25      abuse    0.002156    2.040238  Topic30  -7.1989   0.6326
26    account    0.002156    6.156625  Topic30  -7.1989  -0.4718
27      again    0.002156    3.439991  Topic30  -7.1989   0.1102
28      allow    0.002156    3.310919  Topic30  -7.1989   0.1485
29    browser    0.002156    4.024105  Topic30  -7.1989  -0.0466

[1454 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
24        2  0.149880    able
24        3  0.449640    able
24        4  0.224820    able
24       14  0.149880    able
812       7  0.670176   about
...     ...       ...     ...
426       1  0.418634    year
426       2  0.139545    year
426       5  0.348861    year
926      18  0.823694     yet
235      15  0.515023  zipped

[1613 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 14, 29, 19, 5, 15, 12, 30, 9, 13, 27, 23, 10, 22, 26, 20, 18, 1, 28, 11, 16, 2, 7, 21, 17, 24, 8, 4, 25])